In [ ]:
!pip install langgraph-supervisor langchain-openai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

In [ ]:
# Initialize the AzureChatOpenAI model
model = ChatOpenAI(
    api_key="**********************",
    model="gpt-4o-mini"
)

In [ ]:
model("what is 23*34*45")

AIMessage(content='23 * 34 * 45 = 34,410.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 15, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'stop', 'logprobs': None}, id='run-10c68cc3-19db-44d7-b72c-845720ac4843-0', usage_metadata={'input_tokens': 15, 'output_tokens': 14, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
model(messages=[
    { "role" : "user",
    "content": "What is chemistry?"
                }])

AIMessage(content='Chemistry is the branch of physical science that studies the properties, composition, structure, and changes of matter. It explores how substances interact, combine, and change, and it seeks to understand the underlying principles that govern these processes. Chemistry is often referred to as the "central science" because it connects and overlaps with other scientific disciplines, including biology, physics, environmental science, and material science.\n\nKey areas of chemistry include:\n\n1. **Organic Chemistry**: The study of carbon-containing compounds and their properties, reactions, and synthesis.\n2. **Inorganic Chemistry**: The study of inorganic compounds, which include metals, minerals, and organometallics.\n3. **Physical Chemistry**: The examination of how matter behaves on a molecular and atomic level, including energy transformations and reaction mechanisms.\n4. **Analytical Chemistry**: Focused on the techniques and methods used to separate, identify, an

In [ ]:
!pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.2 MB/s eta 0:00:00


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun #Websearch

In [ ]:
def search_duckduckgo(query: str):
    """Searches DuckDuckGo using LangChain's DuckDuckGoSearchRun tool."""
    search = DuckDuckGoSearchRun()
    return search.invoke(query)

In [ ]:
result =  search_duckduckgo("What is AI Agent?")
print(result)

No good DuckDuckGo Search Result was found


In [ ]:
def add(a: float, b: float):
  """Adds two numbers together.

  Args:
    a: The first number.
    b: The second number.

  Returns:
    The sum of a and b.
  """
  return a+b

def multiply(a: float, b: float):
  """Multiplies two numbers together.

  Args:
    a: The first number.
    b: The second number.

  Returns:
    The product of a and b.
  """
  return a*b

In [ ]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name = "maths_expert",
    prompt="you are a maths expert. Always use one tool at a time"
)

In [ ]:
research_agent = create_react_agent(
    model= model,
    tools=[search_duckduckgo],
    name = "research_expert",
    prompt="you are a research expert with access to websearch. Do not do any Math"
)

In [ ]:
#Create Supervisor workflow
work_flow = create_supervisor(
    [research_agent, math_agent] ,
    model = model,
    prompt=("You are team SUpervisor managing a research expert and maths_expert."
            "for current events and research, use research_expert."
            "for math problem, use math_agent."
    )
)

In [ ]:
#compile and run
app = work_flow.compile()

In [ ]:
result = app.invoke({
    "messages" : [
        {
        "role" : "user",
        "content" : "what is addition of 4 and 6 and answer multipled by 5?"
        }
        ]
})

In [ ]:
result

{'messages': [HumanMessage(content='what is addition of 4 and 6 and answer multipled by 5?', additional_kwargs={}, response_metadata={}, id='6ef6e319-7834-42aa-a2c1-f015fa2c911a'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_J3ftyWctov2L26C65FW4iU0j', 'function': {'arguments': '{}', 'name': 'transfer_to_maths_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 108, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-a8b57f01-1988-412a-85f7-d50c6445c1f0-0', tool_calls=[{'name': 'transfer_to_maths_expert', 'args': {}, 'id': 'call_J3ftyWctov2L26C65FW4iU0j', 'type'

In [ ]:
for m in result['messages']:
  m.pretty_print()

================================ Human Message =================================

what is addition of 4 and 6 and answer multipled by 5?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_maths_expert (call_J3ftyWctov2L26C65FW4iU0j)
 Call ID: call_J3ftyWctov2L26C65FW4iU0j
  Args:
================================= Tool Message =================================
Name: transfer_to_maths_expert

Successfully transferred to maths_expert
================================== Ai Message ==================================
Name: maths_expert

The addition of 4 and 6 is 10, and when multiplied by 5, the result is 50.
================================== Ai Message ==================================
Name: maths_expert

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervisor (7f8f29ab-b8bf-4d49-822d-c0aa92a296ec)
 Call ID: 7f8f29ab-b8bf-4d49-822d-c0aa92a296ec
  Args:
================================= Tool Message 

In [ ]:
result

{'messages': [HumanMessage(content='what is addition of 4 and 6 and answer multipled by 5?', additional_kwargs={}, response_metadata={}, id='6ef6e319-7834-42aa-a2c1-f015fa2c911a'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_J3ftyWctov2L26C65FW4iU0j', 'function': {'arguments': '{}', 'name': 'transfer_to_maths_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 108, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-a8b57f01-1988-412a-85f7-d50c6445c1f0-0', tool_calls=[{'name': 'transfer_to_maths_expert', 'args': {}, 'id': 'call_J3ftyWctov2L26C65FW4iU0j', 'type'